In [6]:
import numpy as np
import copy

In [7]:
class Matrix:
    def __init__(self, matrix):
        self.m = matrix.shape[0]
        self.n = matrix.shape[1]
        self.not0s = [(i, j, matrix[i][j]) for i in range(self.m) for j in range(self.n) if matrix[i][j] != 0]
        
    def T(self):
        other = copy.deepcopy(self)
        other.not0s = []
        for (i, j, x) in self.not0s:
            other.not0s.append((j, i, x))
        return other
            
    def to_np(self):
        result = np.zeros((self.m, self.n))
        for (i, j, x) in self.not0s:
                result[i, j] = x
        return result

    def __matmul__(self, other):
        assert self.n == other.shape[0]
        result = np.zeros(self.m)
        for (i, j, x) in self.not0s:
            result[i] += x * other[j]
        return result

    def __rmatmul__(self, other):
        assert self.m == other.shape[0]
        result = np.zeros(self.n)
        for (i, j, x) in self.not0s:
            result[j] += x * other[i]
        return result

In [8]:
testmatrix = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
assert np.all(Matrix(testmatrix).to_np() == testmatrix)
assert np.all(Matrix(testmatrix).T().to_np() == testmatrix.T)

Минимизировать $\lVert Ax - b \rVert$ это тоже самое, что минимизировать $\lVert Ax - b \rVert^2$, т.к. норма всегда не отрицательна.

Второе выражение есть ни что иное, как $x^TA^TAx - 2b^TAx + const$. Минимизировать неконстантную часть - это тоже самое, что было на лекции, если сделать замену $A = A^TA, b = A^Tb$

In [9]:
def conjugate_gradient(A, b, x_0 = None):
    if x_0 is None:
        x_0 = np.zeros(len(b))
    x = x_0
    
    b = A.T() @ b
    A = Matrix(A.T().to_np() @ A.to_np())
    
    v = (A @ x - b)
    d = v
    v_norm = np.dot(v, v)
    for i in range(len(b)):
        Ad = A @ d
        alpha = v_norm / np.dot(d, Ad)
        x = x - alpha * d
        v = v - alpha * Ad
        v_norm_new = np.dot(v, v)
        d = v + (v_norm_new / v_norm) * d
        v_norm = v_norm_new
    return x

In [10]:
import cvxpy as cp
from tqdm import tqdm

for i in tqdm(range(10)):
    A = np.random.rand(5, 5)
    b = np.random.rand(5)

    x = cp.Variable(5)
    cs = []
    d = cp.Minimize(cp.sum_squares(A @ x - b))
    cp.Problem(d, cs).solve()
    cp_result = np.array(x.value)
    
    result = conjugate_gradient(Matrix(A), b)
    try:
        assert(np.all(result - cp_result < 1e-2))
        assert(np.all(-result + cp_result > -1e-2))
    except:
        print(A)
        print(b)
        print(cp_result)
        print(result)

100%|██████████████████████████████████████████| 10/10 [00:00<00:00, 139.55it/s]
